In [1]:
import os
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

In [2]:
item_cat = pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")
items = pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
shops = pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv")
sample_submission = pd.read_csv("../input/competitive-data-science-predict-future-sales/sample_submission.csv")

In [3]:
df_train = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
df_train

In [4]:
df_train.info()

In [5]:
df_train.isnull().sum()

In [6]:
df_train.describe()

In [7]:
# Converting Date Format
df_train['date'] = pd.to_datetime(df_train['date'],format = '%d.%m.%Y')

In [8]:
# Creating Pivot Table
dataset = df_train.pivot_table(index= ['shop_id','item_id'],values = ['item_cnt_day'],columns = ['date_block_num'],fill_value = 0,aggfunc = 'sum')
dataset.reset_index(inplace=True)
dataset

In [9]:
df_test = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")
df_test

In [10]:
dataset = pd.merge(df_test,dataset,on = ['item_id','shop_id'],how='left' )

In [11]:
dataset.isnull().sum().sum()

In [12]:
dataset.fillna(0,inplace = True)
dataset.isnull().sum()

In [13]:
dataset.info()

In [14]:
dataset.drop(['shop_id','item_id','ID'],inplace=True,axis=1)
dataset

# Split Dataset

In [15]:
X_train = np.expand_dims(dataset.values[:,:-1],axis=2)
y_train = dataset.values[:,-1:]

X_test = np.expand_dims(dataset.values[:,1:],axis=2)
print(X_train.shape,y_train.shape,X_test.shape)

# Creating Sequential Model

In [19]:
my_model = Sequential()
my_model.add(LSTM(units = 64, input_shape = (33,1)))
my_model.add(Dropout(0.4))
my_model.add(Dense(1))
my_model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
my_model.summary()

In [21]:
# Fitting Model
my_model.fit(X_train,y_train,batch_size = 4096, epochs = 10)

In [ ]:
# Creating Submission Model

In [23]:
submission_file = my_model.predict(X_test)
submission_file = submission_file.clip(0,20)
submission_trp = pd.DataFrame({'ID':df_test['ID'],'item_cnt_month':submission_file.ravel()})
submission_trp.to_csv('submission.csv',index = False)
submission_trp

In [26]:
pip install pycaret-ts-alpha